# PHYS 3250 - Spring with Cart and Pendulum (Taylor 7.31)

From the Lagrangian and using Sympy, the equations of motion were determined to be

$$
    \ddot{x} =
    \frac{M g \sin(2\phi)/2 + M l \dot{\phi}^2 \sin \phi - k x}
    {m + M\sin^2 \phi}
$$

and

$$
    \ddot{\phi} = \frac{k x \cos \phi - (m+M)g\sin \phi - M l \dot{\phi}^2 \sin(2\phi)}
    {\left( m + M \sin^2 \phi \right)l}
$$

## Imports

In [ ]:
# Import Panel
import panel as pn
pn.extension('katex', 'mathjax')
pn.config.throttled = True

In [ ]:
# Python Imports
from IPython.display import display

# Numerical
import numpy as np
from scipy.integrate import solve_ivp as sivp
from scipy.integrate._ivp.ivp import OdeResult
from numba import njit

# GUI
import param

# Plotting
from bokeh.io import curdoc
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot, row
from bokeh.models import Rect, Range1d, DataSource as DS, ColumnDataSource as CDS
from bokeh.models.formatters import PrintfTickFormatter

## Notebook Setup

In [ ]:
# Plot Height
FIG_HEIGHT = 5*135

# Golden Ratio
GOLDEN_RATIO = (1 + np.sqrt(5))/2

## Functions

### Interactivity Controls

In [ ]:
# System Equation of Motion
@njit(nogil=True)
def eom(t, X, m, M, g, k, L):

    # Unpack Inputs
    x, xD, ph, phD = X

    # Get sin phi
    sinPhi  = np.sin(ph)
    sin2Phi = np.sin(2*ph)

    # Get Numerator LCM
    mu = m + M*sinPhi**2

    # Get x double dot
    xDD  = M*g*sin2Phi/2 + M*L*phD**2*sinPhi - k*x
    xDD /= mu

    # Get phi double dot
    phDD  = k*x*np.cos(ph) - (m+M)*g*sinPhi - M*L*phD**2*sin2Phi/2
    phDD /= mu*L

    # Return
    return [xD, xDD, phD, phDD]

# Jacobian
@njit(nogil=True)
def jac(t, X, m, M, g, k, L):

    # Unpack Inputs
    x, xD, ph, phD = X

    # Get Commonly Used Terms
    sinPhi  = np.sin(ph)
    sin2Phi = np.sin(2*ph)
    sinPhi2 = sinPhi**2
    cosPhi  = np.cos(ph)
    cos2Phi = np.cos(2*ph)
    cosPhi2 = cosPhi**2
    phD2    = phD**2
    
    # Get Numerator LCM
    mu = m + M*sinPhi**2

    # Indiv Pieces
    dxDDdPhi  = L*M*phD2/4*(np.cos(3*ph) - cosPhi)
    dxDDdPhi += L*m*cosPhi*phD2
    dxDDdPhi += M*g*(cos2Phi - 1)/2
    dxDDdPhi += m*g*cos2Phi
    dxDDdPhi += k*x*sin2Phi
    dxDDdPhi *= M/mu**2

    dpDDdPhi  = M**2*L*phD2*sinPhi2
    dpDDdPhi += M*m*L*phD2*(1 - 2*cosPhi2)
    dpDDdPhi += M*g*cosPhi*(M*sinPhi2 - m*cosPhi2)
    dpDDdPhi -= m**2*g*cosPhi
    dpDDdPhi -= (M+m)*k*x*sinPhi
    dpDDdPhi -= m**2*g*cosPhi
    dpDDdPhi /= L*mu**2

    # Setup the Output
    out = np.zeros((4, 4))
    out[0, 1] = 1
    out[1, 0] = -k/mu
    out[1, 2] = dxDDdPhi
    out[1, 3] = 2*M*L*sinPhi*phD/mu
    out[2, 3] = 1
    out[3, 0] = k*cosPhi/(mu*L)
    out[3, 2] = dpDDdPhi
    out[3, 3] = -2*M*sin2Phi*phD/(M+2*m-M*np.cos(2*ph))

    # Return
    return out

## Classes

### Widgets

In [ ]:
# Widget Classes
class MassSlider(pn.widgets.DiscreteSlider):
    @property
    def labels(self):  # Because Panel was not rewriting the param values that were printed with the formatter str
        return [f'{self.name}: <b>{v:.3f} kg</b>' for v in self.values]

class SpringSlider(pn.widgets.DiscreteSlider):
    @property
    def labels(self):  # Because Panel was not rewriting the param values that were printed with the formatter str
        return [f'{self.name}: <b>{v:.3f} N/m</b>' for v in self.values]

### Parameter Overloads

In [ ]:
# Positive Number
class PositiveNumber(param.Number):

    # Overload Init
    def __init__(self, **params):
        params['bounds'] = params.get('bounds', (0, None))
        params['inclusive_bounds'] = params.get('inclusive_bounds', (False, False))
        super().__init__(**params)

### Parameterized Classes

In [ ]:
class InitialConditions(param.Parameterized):

    # Parameters
    x0 = param.Number(default=1, softbounds=(-1.5, 1.5), step=0.01, label='Initial Position')
    v0 = param.Number(default=0, softbounds=(-2, 2), step=0.01, label='Initial Velocity')
    phi0 = param.Number(default=0, bounds=(-180, 180), step=0.5, label='Initial Angle')
    omega0 = param.Number(default=0, softbounds=(-10, 10), step=0.1, label='Initial Angular Velocity')

    # Data Tuple
    @property
    def tuple(self):
        return (
            self.x0, self.v0, np.deg2rad(self.phi0), np.deg2rad(self.omega0)
        )

    # Parameters List
    def get_params_labels(self):
        out = list(self.param.values().keys())
        out.remove('name')
        return out

    # The Panel
    @property
    def panel(self):
        return pn.Param(self.param, widgets={
            'x0': {'format': PrintfTickFormatter(format='%.2f m'), 'width': 360},
            'v0': {'format': PrintfTickFormatter(format='%.2f m/s'), 'width': 360},
            'phi0': {'format': PrintfTickFormatter(format='%.1f deg'), 'width': 360},
            'omega0': {'format': PrintfTickFormatter(format='%.1f deg/s'), 'width': 360}
        })

In [ ]:
class SimulationParameters(param.Parameterized):

    # Parameters
    cartMass = param.Selector(default=1, objects=np.logspace(-1, 1, 51).tolist(), label='Cart Mass')
    bobMass  = param.Selector(default=1, objects=np.logspace(-1, 1, 51).tolist(), label='Bob Mass')
    gravity  = PositiveNumber(default=1, softbounds=(0.1, 10), step=0.1)
    springConstant = param.Selector(default=1, objects=np.logspace(-1, 1, 51).tolist(), label='Spring Constant')
    pendLength = PositiveNumber(default=1, softbounds=(0.1, 2), step=0.01, label='Pendulum Length')
    maxTime = PositiveNumber(default=20, softbounds=(1, 100), step=1, label='Max Time')
    timeSteps = param.Integer(default=501, bounds=(11, 2001), step=10, label='Time Steps')

    # Arguments Tuple
    @property
    def argtuple(self):
        return (self.cartMass, self.bobMass, self.gravity, self.springConstant, self.pendLength)

    # Parameters List
    def get_params_labels(self):
        out = list(self.param.values().keys())
        out.remove('name')
        return out

    # Time Span (for ODE)
    @property
    def t_span(self):
        return [0, self.maxTime]

    # Time Evaluation Points (for ODE)
    @property
    def t_eval(self):
        return np.linspace(0, self.maxTime, self.timeSteps)

    # The Panel
    @property
    def panel(self):
        return pn.Param(self.param, widgets={
            'cartMass': {'widget_type': MassSlider, 'width':360},
            'bobMass':  {'widget_type': MassSlider, 'width':360},
            'gravity': {'format': PrintfTickFormatter(format='%.1f m/s/s'), 'width':360},
            'springConstant': {'widget_type': SpringSlider, 'width':360},
            'pendLength': {'format': PrintfTickFormatter(format=r'%.2f m'), 'width':360},
            'maxTime': {'width': 360},
            'timeSteps': {'width': 360}
        })

In [ ]:
class Simulation(param.Parameterized):
    # Data
    sol: OdeResult

    # Initial Conditions
    initialConds = InitialConditions(name='Initial Conditions')
    
    # Widgets
    widgets = SimulationParameters(name='Parameters')
    time = param.Integer(default=0, bounds=(0, widgets.timeSteps-1))

    # Init
    def __init__(self, **params):

        # Pass the Everything to Super
        super().__init__(**params)

        # Initialize some Figs
        self.figures = {
            'xt': figure(x_range=(0, self.widgets.maxTime)),
            'pt': figure(x_range=(0, self.widgets.maxTime)),
            'xPhase': figure(),
            'pPhase': figure(),
            'ani': figure(
                height=FIG_HEIGHT, width=int(GOLDEN_RATIO*FIG_HEIGHT),
                match_aspect=True, aspect_scale=1,
                x_range=(-2.25, 2.25), y_range=(-2.25, 0.25)
            )
        }
        self.figures['phase'] = gridplot(
            [[self.figures['xt'], self.figures['xPhase']],
             [self.figures['pt'], self.figures['pPhase']]],
            width=int(FIG_HEIGHT/2), height=int(FIG_HEIGHT/2),
            toolbar_location='below'
        )
        self.figPane = pn.pane.Bokeh(row(self.figures['phase'], self.figures['ani']), theme='dark_minimal')
        self.sources = {}

        # Get the Time Player
        self.timePlayer = pn.widgets.Player.from_param(self.param.time, interval=20,width=360)

        # Get Initial Solution
        self._cartLen = 0.2
        self._cartHei = 0.1
        self.__plots_initialized = False
        self._get_solution()
        self.__init_phase()
        self.__init_ani()
        self.__plots_initialized = True

        # Watchers
        self.initialConds.param.watch(
            self._reset_time, self.initialConds.get_params_labels(),
            precedence=0
        )
        self.widgets.param.watch(
            self._reset_time, self.widgets.get_params_labels(),
            precedence=0
        )
        self.widgets.param.watch(self._update_time, ['timeSteps'], precedence=1)
        self.initialConds.param.watch(
            self._get_solution, self.initialConds.get_params_labels(),
            precedence=5
        )
        self.widgets.param.watch(
            self._get_solution, self.widgets.get_params_labels(),
            precedence=5
        )

    # Get the Solution on Updates to the ICs or Widgets
    # @param.depends('initialConds.param', 'widgets.param', watch=True, on_init=True)
    def _get_solution(self, event=None):
        self.sol = sivp(
            eom,
            t_span=self.widgets.t_span,
            t_eval=self.widgets.t_eval,
            y0=self.initialConds.tuple,
            args=self.widgets.argtuple,
            method='LSODA',
            jac=jac
        )
        if self.__plots_initialized:
            self._update_phase(updateLines=True)
            self._update_ani()

    # Initialize Phase Plot
    def __init_phase(self):

        # Setup the Line Sources
        self.sources['xtLine']     = CDS(data=dict(x=self.sol.t, y=self.sol.y[0]))
        self.sources['xPhaseLine'] = CDS(data=dict(x=self.sol.y[0], y=self.sol.y[1]))
        self.sources['ptLine']     = CDS(data=dict(x=self.sol.t, y=self.sol.y[2]))
        self.sources['pPhaseLine'] = CDS(data=dict(x=self.sol.y[2], y=self.sol.y[3]))

        # Setup the Line Sources
        self.sources['xtMark']     = CDS(data=dict(x=[self.sol.t[self.time]], y=[self.sol.y[0][self.time]]))
        self.sources['xPhaseMark'] = CDS(data=dict(x=[self.sol.y[0][self.time]], y=[self.sol.y[1][self.time]]))
        self.sources['ptMark']     = CDS(data=dict(x=[self.sol.t[self.time]], y=[self.sol.y[2][self.time]]))
        self.sources['pPhaseMark'] = CDS(data=dict(x=[self.sol.y[2][self.time]], y=[self.sol.y[3][self.time]]))

        # Plot Results
        self.figures['xt'    ].line('x', 'y', source=self.sources['xtLine'], line_width=2)
        self.figures['xPhase'].line('x', 'y', source=self.sources['xPhaseLine'], line_width=2)
        self.figures['pt'    ].line('x', 'y', source=self.sources['ptLine'], line_width=2)
        self.figures['pPhase'].line('x', 'y', source=self.sources['pPhaseLine'], line_width=2)

        # Plot Time Marker
        self.figures['xt'    ].x('x', 'y', source=self.sources['xtMark'], size=10, color='red', line_width=2)
        self.figures['xPhase'].x('x', 'y', source=self.sources['xPhaseMark'], size=10, color='red', line_width=2)
        self.figures['pt'    ].x('x', 'y', source=self.sources['ptMark'], size=10, color='red', line_width=2)
        self.figures['pPhase'].x('x', 'y', source=self.sources['pPhaseMark'], size=10, color='red', line_width=2)
        
        # Labels
        self.figures['xt'].xaxis.axis_label = r'$$t$$ (s)'
        self.figures['xt'].yaxis.axis_label = r'$$x$$ (m)'
        self.figures['xPhase'].xaxis.axis_label = r'$$x$$ (m)'
        self.figures['xPhase'].yaxis.axis_label = r'$$\dot{x}$$ (m/s)'
        self.figures['pt'].xaxis.axis_label = r'$$t$$ (s)'
        self.figures['pt'].yaxis.axis_label = r'$$\phi$$ (rad)'
        self.figures['pPhase'].xaxis.axis_label = r'$$\phi$$ (rad)'
        self.figures['pPhase'].yaxis.axis_label = r'$$\dot{\phi}$$ (rad/s)'
        pn.io.push_notebook(self.figPane)

    # Update Phase Diagram
    @param.depends('time', watch=True)
    def _update_phase(self, updateLines=False):
        if updateLines:
            self.sources['xtLine'].data     = dict(x=self.sol.t, y=self.sol.y[0])
            self.sources['xPhaseLine'].data = dict(x=self.sol.y[0], y=self.sol.y[1])
            self.sources['ptLine'].data     = dict(x=self.sol.t, y=self.sol.y[2])
            self.sources['pPhaseLine'].data = dict(x=self.sol.y[2], y=self.sol.y[3])
            self.figures['xt'].x_range.end = self.widgets.maxTime
            self.figures['pt'].x_range.end = self.widgets.maxTime
        
        # Update Markers
        self.sources['xtMark'].data     = dict(x=[self.sol.t[self.time]], y=[self.sol.y[0][self.time]])
        self.sources['xPhaseMark'].data = dict(x=[self.sol.y[0][self.time]], y=[self.sol.y[1][self.time]])
        self.sources['ptMark'].data     = dict(x=[self.sol.t[self.time]], y=[self.sol.y[2][self.time]])
        self.sources['pPhaseMark'].data = dict(x=[self.sol.y[2][self.time]], y=[self.sol.y[3][self.time]])
        pn.io.push_notebook(self.figPane)

    # Initialize Phase Plot
    def __init_ani(self):

        # Cart Size
        cartLen = self._cartLen
        cartHei = self._cartHei
        
        # Other Important Plot Things
        xMin = self.figures['ani'].x_range.start
        
        # Spring Argument
        sprRad     = cartHei / 2
        nTurns     = 40
        nSpringPts = 501
        ptPad      = 40
        sprLen     = self.sol.y[0][0] - xMin
        w          = np.linspace(0, sprLen, nSpringPts)
        xSpr       = w + xMin
        ySpr       = sprRad*np.ones_like(w)
        ySpr[ptPad:-ptPad] = sprRad * (1 + np.sin(2*np.pi * nTurns * w[ptPad:-ptPad] / sprLen)/2)
        
        # Create Sources
        self.sources['cart'] = CDS(dict(x=[cartLen/2+self.sol.y[0][0]]))
        self.sources['string'] = CDS(dict(
            x=[cartLen/2+self.sol.y[0][0], self.sol.y[0][0] + cartLen/2+self.widgets.pendLength*np.sin(self.sol.y[2][0])],
            y=[cartHei/2, cartHei/2 - self.widgets.pendLength*np.cos(self.sol.y[2][0])]
        ))
        self.sources['bob'] = CDS(dict(
            x=[cartLen/2+self.sol.y[0][0] + self.widgets.pendLength*np.sin(self.sol.y[2][0])],
            y=[cartHei/2 - self.widgets.pendLength*np.cos(self.sol.y[2][0])]
        ))
        self.sources['spring'] = CDS(dict(x=xSpr, y=ySpr))

        # Initialize Artists
        self.figures['ani'].hspan(y=[0], line_width=1.25, color='black')
        r = Rect(x='x', y=cartHei/2, width=cartLen, height=cartHei, fill_color='black')
        self.figures['ani'].add_glyph(self.sources['cart'], r)
        self.figures['ani'].line('x', 'y', source=self.sources['string'], line_width=1.5, color='#66ccff')
        self.figures['ani'].circle('x', 'y', source=self.sources['bob'], size=16, fill_color='orange', line_color='black')
        self.figures['ani'].line('x', 'y', source=self.sources['spring'], color='black')
        pn.io.push_notebook(self.figPane)

    # Update Animation
    @param.depends('time', watch=True)
    def _update_ani(self):
        
        # Get Locals
        cartLen, cartHei = self._cartLen, self._cartHei
        L = self.widgets.pendLength
        
        # Update Sources
        self.sources['cart'].data = dict(x=[cartLen/2+self.sol.y[0][self.time]])
        self.sources['string'].data = dict(
            x=[cartLen/2+self.sol.y[0][self.time], cartLen/2+self.sol.y[0][self.time] + L*np.sin(self.sol.y[2][self.time])],
            y=[cartHei/2, cartHei/2 - L*np.cos(self.sol.y[2][self.time])]
        )
        self.sources['bob'].data = dict(
            x=[cartLen/2+self.sol.y[0][self.time] + L*np.sin(self.sol.y[2][self.time])],
            y=[cartHei/2 - L*np.cos(self.sol.y[2][self.time])]
        )
        
        # Update Spring
        sprLen     = self.sol.y[0][self.time] - self.figures['ani'].x_range.start
        w          = np.linspace(0, sprLen, 501)
        xSpr       = w + self.figures['ani'].x_range.start
        self.sources['spring'].data['x'] = xSpr
        
        # Push Updates
        pn.io.push_notebook(self.figPane)

    # Reset Time
    # @param.depends('initialConds.param', 'widgets.param', watch=True)
    def _reset_time(self, event):
        self.time = 0

    # Update Valid Times
    # @param.depends('widgets.timeSteps', watch=True)
    def _update_time(self, event):
        self.param.time.bounds = (0, self.widgets.timeSteps-1)

    @param.depends('widgets.maxTime', 'widgets.timeSteps', watch=True)
    def _update_interval(self):
        self.timePlayer.interval = int(1000*self.widgets.maxTime/(self.widgets.timeSteps - 1))

    # The Panel
    @property
    def panel(self):
        return pn.Row(
            pn.Column(self.initialConds.panel, self.widgets.panel, self.timePlayer),
            self.figPane
        )

In [ ]:
a = Simulation()
a.panel.servable()